In [1]:
#import all the necessary packages here

import numpy as np

import csv

import pandas as pd

import warnings

import sklearn

import os

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

import statsmodels.api as sm
from scipy import stats

warnings.simplefilter('ignore')

In [2]:
#Change the directory to the folder where you have all the raw data

os.chdir(r"C:\Users\Shilp\Documents\Shilpa\Shilpa MSBA\Semester 1\Programming for Analytics [CIS 9650]\Final Group Project")

In [3]:
# Import daily level US - State dataset with TAVG, TMAX, TMIN, PRCP, SNOW variables
us_state_master = pd.read_csv("data_2010_2020.csv", sep=",", error_bad_lines = False)
us_state_master.head()

,State,date,metric_name,metric_value
0,AK,20200101,PRCP,59.407895
1,AK,20200101,SNOW,48.717949
2,AK,20200101,TAVG,-161.350319
3,AK,20200101,TMAX,-94.241935
4,AK,20200101,TMIN,-171.959677


In [35]:
us_state_master_pivot = pd.pivot_table(us_state_master, values ='metric_value', index =['State','date'], columns =['metric_name']) 

In [36]:
us_state_master_pivot.reset_index(inplace=True)
us_state_master_pivot['date'] = us_state_master_pivot['date'].astype(int)
us_state_master_pivot.head()

metric_name,State,date,PRCP,SNOW,TAVG,TMAX,TMIN
0,AK,20100101,0.617347,0.609375,-180.643357,-133.130435,-212.507246
1,AK,20100102,1.077320,1.656000,-195.906475,-158.857143,-224.400735
2,AK,20100103,15.938462,22.937008,-160.553957,-134.227941,-203.400735
3,AK,20100104,17.061224,10.875000,-119.578571,-94.698182,-160.160000
4,AK,20100105,9.020408,3.156250,-89.428571,-71.727273,-123.640000


In [96]:
us_state_master_pivot['Year'] = us_state_master_pivot['date'].astype(str).str[0:4]

In [135]:
us_state_master_pivot_v2 = us_state_master_pivot[us_state_master_pivot.Year == '2010'] 
segmentation_sample_2010 = us_state_master_pivot_v2.groupby('State').aggregate({'PRCP': 'mean',
                             'SNOW': 'mean','TAVG': 'mean','TMIN': 'mean','TAVG': 'mean'}).fillna(us_state_master_pivot_v2.mean()).reset_index()

In [148]:
segmentation_sample_2010_v2  = segmentation_sample_2010.drop(columns =['State'])
segmentation_sample_2010_v2.head()

,PRCP,SNOW,TAVG,TMIN
count,51.000000,51.000000,51.000000,51.000000
mean,28.488803,2.482587,108.401068,53.834306
std,9.757530,1.731010,50.554044,44.796639
min,11.242227,0.000000,-6.952798,-44.342785
25%,22.261193,1.103041,70.470141,24.202682
50%,30.276541,2.600036,108.400799,53.834306
75%,33.490059,3.652770,142.782291,79.404258
max,71.650000,6.580243,205.393617,174.500980


In [151]:
cols_of_interest = ['PRCP', 'SNOW']
data = segmentation_sample_2010_v2[cols_of_interest]
data.head()

,PRCP,SNOW
count,51.000000,51.000000
mean,28.488803,2.482587
std,9.757530,1.731010
min,11.242227,0.000000
25%,22.261193,1.103041
50%,30.276541,2.600036
75%,33.490059,3.652770
max,71.650000,6.580243


In [141]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(data)

In [142]:
#Set number of clusters at initialisation time
k_means = KMeans(n_clusters=5)
#Run the clustering algorithm
model = k_means.fit(X)
model
#Generate cluster predictions and store in y_hat
y_hat = k_means.predict(X)

In [143]:
from sklearn import metrics
labels = k_means.labels_
metrics.silhouette_score(X, labels, metric = 'euclidean')
metrics.calinski_harabasz_score(X, labels)

49.43512368153119

In [156]:
segmentation_sample_2010['Cluster'] = pd.Series(y_hat)

In [157]:
segmentation_sample_2010

,State,PRCP,SNOW,TAVG,TMIN,Cluster
0,AK,27.339963,5.049815,-6.952798,-44.342785,2
1,AL,36.162263,0.200902,165.978721,104.139113,1
2,AR,27.872713,0.597835,153.499726,100.533676,1
3,AZ,11.451286,1.264945,116.826111,70.068396,4
4,CA,25.961835,1.488500,120.666589,70.479295,1
5,CO,13.451427,4.641145,40.033634,-9.777973,4
6,CT,36.111758,2.604263,108.400799,50.452244,0
7,DC,71.650000,0.000000,108.400799,53.834306,3
8,DE,30.847800,3.573659,140.520548,84.250000,0
9,FL,35.927104,0.000000,205.393617,156.429086,1
